In [ ]:
! cd /content/drive/MyDrive/Colab\ Notebooks && git clone https://github.com/eriklindernoren/PyTorch-YOLOv3.git

In [ ]:
! pip install terminaltables imgaug=="0.4.0"

In [ ]:
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/PyTorch-YOLOv3')

# download pre-trained weights

In [ ]:
! cd weights && sh download_weights.sh

# parameter settings

In [ ]:
import numpy as np
from PIL import Image
import torch
from torch.autograd import Variable
import torchvision.transforms as transforms

from pytorchyolo.utils.transforms import Resize, DEFAULT_TRANSFORMS

In [ ]:
model_path = 'config/yolov3.cfg'
weights_path = 'weights/yolov3.weights'
classes = 'data/coco.names'
output = 'output'
batch_size = 1
img_size = 416
n_cpu = 8
conf_thres = 0.5
nms_thres = 0.4

# prepare image data

In [ ]:
img_path = 'data/samples/dog.jpg'

img = np.array(Image.open(img_path).convert('RGB'), dtype=np.uint8)
boxes = np.zeros((1, 5))
transform=transforms.Compose([DEFAULT_TRANSFORMS, Resize(img_size)])

img, _ = transform((img, boxes))

In [ ]:
def tensor2im(img_tensor):
    # dimension: CHW -> HWC
    img = img_tensor.numpy().transpose(1, 2, 0)
    img = np.asarray(img*255, dtype=np.uint8)
    # ensure memory contiguity
    return np.ascontiguousarray(img)

Image.frombuffer('RGB', (img_size, img_size), tensor2im(img))

# extract `detect_directory` function and detect objects step by step

In [ ]:
from pytorchyolo.models import load_model
from pytorchyolo.utils.utils import load_classes, rescale_boxes, non_max_suppression

In [ ]:
classes = load_classes(classes)
model = load_model(model_path, weights_path)

In [ ]:
model

In [ ]:
model.eval()

Tensor = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor

input_img = Variable(img.type(Tensor))
input_img = torch.unsqueeze(input_img, 0)

with torch.no_grad():
    detections = model(input_img)
    detections = non_max_suppression(detections, conf_thres, nms_thres)
    if len(detections) == 1:
        detections = detections[0]

# Visualization for detection results

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import random

# Create plot
img = np.array(Image.open(img_path))
plt.figure()
fig, ax = plt.subplots(1)
ax.imshow(img)
# Rescale boxes to original image
detections = rescale_boxes(detections, img_size, img.shape[:2])
unique_labels = detections[:, -1].cpu().unique()
n_cls_preds = len(unique_labels)
# Bounding-box colors
cmap = plt.get_cmap("tab20b")
colors = [cmap(i) for i in np.linspace(0, 1, n_cls_preds)]
bbox_colors = random.sample(colors, n_cls_preds)
for x1, y1, x2, y2, conf, cls_pred in detections:

    print(f"\t+ Label: {classes[int(cls_pred)]} | Confidence: {conf.item():0.4f}")

    box_w = x2 - x1
    box_h = y2 - y1

    color = bbox_colors[int(np.where(unique_labels == int(cls_pred))[0])]
    # Create a Rectangle patch
    bbox = patches.Rectangle((x1, y1), box_w, box_h, linewidth=2, edgecolor=color, facecolor="none")
    # Add the bbox to the plot
    ax.add_patch(bbox)
    # Add label
    plt.text(
        x1,
        y1,
        s=classes[int(cls_pred)],
        color="white",
        verticalalignment="top",
        bbox={"color": color, "pad": 0})